In [14]:
# installing missing libraries and updating existing ones
!pip install transformers
!pip install datasets
!pip install --upgrade pandas
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
# importing libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import evaluate
from transformers import DataCollatorWithPadding
import warnings

In [16]:
# parameters used in the code
data_path = "/kaggle/input/method-singlelbl-subclass-csv/method_singlelbl_subclass.csv"
text_column_name = "Combined_text"
label_column_name = "label"
model_name = "microsoft/Multilingual-MiniLM-L12-H384"
test_size = 0.2
num_labels = 6 

In [17]:
# loading the data sample and generating input and output columns
df = pd.read_csv(data_path)
df['Combined_text'] = df.apply(lambda row: ' '.join([str(row['Prompt']), str(row['Reply'])]), axis=1)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df[label_column_name] = le.fit_transform(df['Subclass'].tolist())

In [18]:
# splitting the data into train-test
df_train, df_test = train_test_split(df, test_size=test_size)
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

In [19]:
# tokenization for the hugging face trasformer models
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(examples):
    return tokenizer(examples[text_column_name], padding="max_length", truncation=True, max_length=512)
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)
print(tokenized_test)

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset({
    features: ['Prompt', 'Reply', 'Subclass', 'Combined_text', 'label', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 818
})


In [20]:
# converting the label column to tensors
tokenized_train = tokenized_train.map(lambda x: {"label": torch.tensor(x["label"]).long()})
tokenized_test = tokenized_test.map(lambda x: {"label": torch.tensor(x["label"]).long()})

Map:   0%|          | 0/3270 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [21]:
# initializing the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
# setting up evaluation metrics
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
# training arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=1e-5, # lowering the learning rate
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    report_to=[]
)

In [24]:
# setting up the trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [25]:
# training the model using train data
warnings.filterwarnings('ignore')
trainer.train()
trainer.save_model('Multilingual-MiniLM-L12-H384')

Epoch,Training Loss,Validation Loss,Accuracy
1,1.536500,1.388067,0.438875
2,1.359400,1.266714,0.606357
3,1.219800,1.090439,0.715159
4,1.039900,0.940318,0.778729
5,0.879600,0.815546,0.837408
6,0.767400,0.721117,0.853301
7,0.664300,0.645593,0.864303
8,0.587000,0.580496,0.881418
9,0.522000,0.586192,0.872861
10,0.479000,0.513355,0.876528


In [26]:
# evaluating the model on the test data
preds = trainer.predict(tokenized_test)
pred_labels = np.argmax(preds.predictions, axis=1)
truth = df_test['label'].tolist()
print(classification_report(truth, pred_labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.90      0.98      0.94        47
           2       0.95      0.94      0.95       359
           3       0.25      0.09      0.13        11
           4       0.89      0.96      0.92       140
           5       0.93      0.94      0.93       257

    accuracy                           0.93       818
   macro avg       0.65      0.65      0.65       818
weighted avg       0.92      0.93      0.92       818

